In [ ]:
# !pip install demoji

In [ ]:
# !pip install textblob

In [ ]:
# !pip install afinn

In [ ]:
# !pip install wordcloud

In [ ]:
# !pip install textblob-fr

In [ ]:
# !pip install plotly

In [1]:
import pandas as pd
import numpy as np
import plotly
import matplotlib
import plotly.express as px
import requests
import seaborn as sns
from matplotlib import pyplot as plt
import datetime
import searchtweets
from textblob import TextBlob
from afinn import Afinn
from datetime import *
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from textblob import TextBlob, Blobber
from textblob_fr import PatternTagger, PatternAnalyzer
import re
from nltk.corpus import stopwords
import nltk
import demoji
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [2]:
# nltk.download('stopwords')
# nltk.download('punkt')

# Introduction

# Data Wrangling

In [3]:
df_tweets = pd.read_csv("~/work/projetpy/csv_tweets/tweets_vaccins.csv")

Un même tweet peut être posté par le même auteur à des heures différentes. La cellule ci-dessous permet de supprimer ces tweets dupliqués qui ont des valeurs communes de *texte* et *type_vaccin*. Ceratins tweets citent plus d'un vaccin, leurs repétitions selon *texte* et *type_vaccin* est légitime et ne seront pas pris en compte dans la suppression.

In [4]:
df_tweets["datetime"] = df_tweets["date"].apply(lambda x : datetime.strptime(x, "%Y-%m-%d %H:%M:%S"))

df_tweets["date"] = df_tweets["datetime"].apply(datetime.date)

In [5]:
df_tweets.drop_duplicates(subset = ["texte", "type_vaccin"], keep = 'first', inplace=True, ignore_index = True)

In [6]:
df_tweets.head(10)

,date,id,nbre_like,nbre_retweet,screenName,texte,type,type_vaccin,datetime
0,2021-03-01,1366184519662006274,1,0,yokko_mars,"@silvano_trotta Au Japon, on refuse le don de ...",tweet,pfizer,2021-03-01 00:32:26
1,2021-03-01,1366299423781892098,2,0,Epcocorico,@France_Dessinee @Pfizer_France Tragique,tweet,pfizer,2021-03-01 08:09:01
2,2021-03-01,1366307752096768000,2,0,socarre,@thierrycogne @dc_morel Vaccinons en pharmacie...,tweet,janssen,2021-03-01 08:42:07
3,2021-03-01,1366307752096768000,2,0,socarre,@thierrycogne @dc_morel Vaccinons en pharmacie...,tweet,moderna,2021-03-01 08:42:07
4,2021-03-01,1366307752096768000,2,0,socarre,@thierrycogne @dc_morel Vaccinons en pharmacie...,tweet,pfizer,2021-03-01 08:42:07
5,2021-03-01,1366314384935550977,5,0,DICKENSDAVID1,@PandaPromeneur 2/2 Pour conclure laissez moi ...,tweet,astrazeneca,2021-03-01 09:08:28
6,2021-03-01,1366338264492310533,29,7,DICKENSDAVID1,"Vous êtes Parisiens, âgés de 50 a 65. et souf...",tweet,astrazeneca,2021-03-01 10:43:22
7,2021-03-01,1366378376999669761,0,0,sandrine_rachez,@olivierveran quand le vaccin Pfizer sera-t-il...,tweet,pfizer,2021-03-01 13:22:45
8,2021-03-01,1366382071409643528,3,1,luna431168,@Drmartyufml Ne pas passer à côté de la territ...,tweet,janssen,2021-03-01 13:37:26
9,2021-03-01,1366386744745164802,0,1,GARCIAJacqueli2,Le vaccin de Pfizer serait moins efficace sur ...,tweet,pfizer,2021-03-01 13:56:00


In [7]:
df_tweets.shape

(6005, 9)

Ainsi, 6005 tweets ont été requètés via l'API sur la période de 01 mars 2021 au 26 décembre 2021.

### Nettoyage des données textuelles

Les tweets sont des objets "sales" et cela rend dificile leur manipulation. Pour la suite, il faudra les nettoyer et la foncttion transforme définie à cet effet sert à supprimer les caractères spéciaux, les émojis, les ponctuations, à tranformer les majuscules en minuscule, etc...

In [8]:
def transforme(texte):
    texte = texte.lower() # mettre les mots en minuscule
    #retirer les liens
    for item in re.compile("http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+").findall(texte):
        texte=texte.replace(item, "")
    # enlever le retour à la ligne
    texte = texte.replace("\n", " ").replace("\r", "") 
    # supprimer ",", "!", "?", "%", "(",")","/",'"', "$","£", "_", "-", "+", "*", "µ", ":","&,"§" 
    texte = re.sub(r"[,\!\?\%\(\)\/\"\$\£\-\+\*\µ,\:\&\§]", " ", texte) 
    # retirer les hashtags #
    for item in re.compile("([#]\w+)").findall(texte):
        texte=texte.replace(item, "")
    # retirer les apostrophes '
    for item in re.compile("([\’])").findall(texte):
        texte=texte.replace(item, " ")
    for item in re.compile("([\'])").findall(texte):
        texte=texte.replace(item, " ")
    # retirer les points de suspension
    for item in re.compile("([.]{1,5})").findall(texte):
        texte=texte.replace(item, "") 
    # retirer les personnes tagées
    for item in re.compile("([@]\w+)").findall(texte):
        texte=texte.replace(item, "")
    # retirer les adresses mail
    for item in re.findall('\S+@\S+', texte) :
        texte=texte.replace(item, "")
    # retire les mots contenant des chiffres
    texte = re.sub(r"[A-Za-z\.]*[0-9]+[A-Za-z%°\.]*", "", texte)
    # retirer les emojis
    for item in demoji.findall(texte):
        texte=texte.replace(item,"")
    return texte

In [9]:
df_tweets["texte_propre"] = df_tweets["texte"].apply(lambda x: transforme(x))
df_tweets.head()

,date,id,nbre_like,nbre_retweet,screenName,texte,type,type_vaccin,datetime,texte_propre
0,2021-03-01,1366184519662006274,1,0,yokko_mars,"@silvano_trotta Au Japon, on refuse le don de ...",tweet,pfizer,2021-03-01 00:32:26,au japon on refuse le don de sang de personn...
1,2021-03-01,1366299423781892098,2,0,Epcocorico,@France_Dessinee @Pfizer_France Tragique,tweet,pfizer,2021-03-01 08:09:01,tragique
2,2021-03-01,1366307752096768000,2,0,socarre,@thierrycogne @dc_morel Vaccinons en pharmacie...,tweet,janssen,2021-03-01 08:42:07,vaccinons en pharmacie et chez les médecins ...
3,2021-03-01,1366307752096768000,2,0,socarre,@thierrycogne @dc_morel Vaccinons en pharmacie...,tweet,moderna,2021-03-01 08:42:07,vaccinons en pharmacie et chez les médecins ...
4,2021-03-01,1366307752096768000,2,0,socarre,@thierrycogne @dc_morel Vaccinons en pharmacie...,tweet,pfizer,2021-03-01 08:42:07,vaccinons en pharmacie et chez les médecins ...


In [10]:
# Pour enlever les stopwords
stop_words = set(stopwords.words('french'))

### Calcul du sentiment d'un tweet à partir de la polarité de Textblob

In [11]:
tb = Blobber(pos_tagger=PatternTagger(), analyzer=PatternAnalyzer())

In [12]:
def tweet_polarite(text):
    return tb(text).sentiment[0]

In [13]:
def tweet_label_car(text):
    if tweet_polarite(text) > 0 :
        return "positif"
    if tweet_polarite(text) < 0 :
        return "negatif"
    return "neutre"

In [14]:
def tweet_label(text):
    if tweet_polarite(text) > 0 :
        return 1
    if tweet_polarite(text) < 0 :
        return -1
    return 0

In [15]:
df_tweets["score_polarite"] = df_tweets["texte_propre"].apply(tweet_polarite)
df_tweets["label_car"] = df_tweets["texte_propre"].apply(tweet_label_car)
df_tweets["label"] = df_tweets["texte_propre"].apply(tweet_label)

In [16]:
df_tweets.head()

,date,id,nbre_like,nbre_retweet,screenName,texte,type,type_vaccin,datetime,texte_propre,score_polarite,label_car,label
0,2021-03-01,1366184519662006274,1,0,yokko_mars,"@silvano_trotta Au Japon, on refuse le don de ...",tweet,pfizer,2021-03-01 00:32:26,au japon on refuse le don de sang de personn...,0.00,neutre,0
1,2021-03-01,1366299423781892098,2,0,Epcocorico,@France_Dessinee @Pfizer_France Tragique,tweet,pfizer,2021-03-01 08:09:01,tragique,-0.40,negatif,-1
2,2021-03-01,1366307752096768000,2,0,socarre,@thierrycogne @dc_morel Vaccinons en pharmacie...,tweet,janssen,2021-03-01 08:42:07,vaccinons en pharmacie et chez les médecins ...,0.15,positif,1
3,2021-03-01,1366307752096768000,2,0,socarre,@thierrycogne @dc_morel Vaccinons en pharmacie...,tweet,moderna,2021-03-01 08:42:07,vaccinons en pharmacie et chez les médecins ...,0.15,positif,1
4,2021-03-01,1366307752096768000,2,0,socarre,@thierrycogne @dc_morel Vaccinons en pharmacie...,tweet,pfizer,2021-03-01 08:42:07,vaccinons en pharmacie et chez les médecins ...,0.15,positif,1


### Création des métadonnées

In [17]:
# Création des 'dummy" variables
dummy_df = pd.get_dummies(df_tweets.type_vaccin, prefix="vaccin", prefix_sep='_', drop_first=False)
df_tweets[dummy_df.columns] = dummy_df

In [18]:
def decompte_hashtags(text):
    exp_match = re.compile("([#]\w+)")
    res = exp_match.findall(text)
    return len(res)

df_tweets["nbre_hashtags"] = df_tweets["texte"].apply(lambda x : decompte_hashtags(x))

In [19]:
def decompte_tags(text):
    exp_match = re.compile("([@]\w+)")
    res = exp_match.findall(text)
    return len(res)

df_tweets["nbre_tags"] = df_tweets["texte"].apply(lambda x : decompte_tags(x))

In [20]:
def decompte_exclamation(text):
    exp_match = re.compile("(\w?\s?[!])")
    res = exp_match.findall(text)
    return len(res)

df_tweets["nbre_exclamation"] = df_tweets["texte"].apply(lambda x : decompte_exclamation(x))

In [21]:
def decompte_mails(text):
    exp_match = re.compile("(\w+[@]\w+[.]\w+)")
    res = exp_match.findall(text)
    return len(res)

df_tweets["nbre_mails"] = df_tweets["texte"].apply(lambda x : decompte_mails(x))

In [22]:
def decompte_interrogation(text):
    exp_match = re.compile("(\w?\s?[?])")
    res = exp_match.findall(text)
    return len(res)

df_tweets["nbre_interrogation"] = df_tweets["texte"].apply(lambda x : decompte_interrogation(x))

In [23]:
def decompte_mot_maj(text):
    exp_match = re.compile("([A-Z][A-Z]+)")
    res = exp_match.findall(text)
    return len(res)

df_tweets["nbre_maj"] = df_tweets["texte"].apply(lambda x : decompte_mot_maj(x))

In [24]:
df_tweets.head()

,date,id,nbre_like,nbre_retweet,screenName,texte,type,type_vaccin,datetime,texte_propre,...,vaccin_astrazeneca,vaccin_janssen,vaccin_moderna,vaccin_pfizer,nbre_hashtags,nbre_tags,nbre_exclamation,nbre_mails,nbre_interrogation,nbre_maj
0,2021-03-01,1366184519662006274,1,0,yokko_mars,"@silvano_trotta Au Japon, on refuse le don de ...",tweet,pfizer,2021-03-01 00:32:26,au japon on refuse le don de sang de personn...,...,0,0,0,1,1,1,0,0,1,2
1,2021-03-01,1366299423781892098,2,0,Epcocorico,@France_Dessinee @Pfizer_France Tragique,tweet,pfizer,2021-03-01 08:09:01,tragique,...,0,0,0,1,0,2,0,0,0,0
2,2021-03-01,1366307752096768000,2,0,socarre,@thierrycogne @dc_morel Vaccinons en pharmacie...,tweet,janssen,2021-03-01 08:42:07,vaccinons en pharmacie et chez les médecins ...,...,0,1,0,0,0,2,0,0,0,1
3,2021-03-01,1366307752096768000,2,0,socarre,@thierrycogne @dc_morel Vaccinons en pharmacie...,tweet,moderna,2021-03-01 08:42:07,vaccinons en pharmacie et chez les médecins ...,...,0,0,1,0,0,2,0,0,0,1
4,2021-03-01,1366307752096768000,2,0,socarre,@thierrycogne @dc_morel Vaccinons en pharmacie...,tweet,pfizer,2021-03-01 08:42:07,vaccinons en pharmacie et chez les médecins ...,...,0,0,0,1,0,2,0,0,0,1


In [25]:
df_tweets.shape

(6005, 23)

In [26]:
# df_tweets.dtypes

Ainsi, 6005 tweets ont pu être requetés et 21 variables construites.

# Analyse de données exploratoire

### Stats descriptives

In [ ]:
df_tweets.groupby("type_vaccin")["id"].count()

### Analyse temporelle

Dans cette partie, nous verrons si nous pouvons explorer des tendances par rapport au temps.   
Premièrement, il est utile de visualiser la distribution temporelle de tous les tweets dans l'ensemble de données.

In [ ]:
plt.figure(figsize=(30, 8))
sns.set_style('darkgrid')
sns.histplot(df_tweets, x="date")
plt.title("Fréquence des tweets par rapport au temps")

In [ ]:
df_tweets.groupby("label_car")["id"].count()

In [ ]:
plt.figure(figsize=(5, 5))
sns.set_style('darkgrid')
sns.histplot(df_tweets, x="label_car",  shrink=0.7)
plt.title("Répartition des tweets par label de sentiment")

Parmi tous les tweets, environ la moitié ont été désignés par TextBlob comme positifs en termes de sentiment (polarité = > 1), l'autre moitié étant constituée de 1203 tweets négatifs (polarité < 0,0) et de 1495 tweets neutres (polarité = 0).

In [ ]:
df_date = df_tweets.groupby(['date']).agg(np.nanmean).reset_index()
df_date['nbre_tweet'] = df_tweets.groupby(['date']).count().reset_index()['id']
df_date = df_date[['date', 'nbre_tweet', 'score_polarite', 'nbre_like']]
df_date["score_polarite"] = df_date["score_polarite"].astype(float)
# df_date['nbre_tweet']

In [ ]:
plt.figure(figsize=(30, 8))
plot = plt.scatter(df_date.index, df_date['nbre_tweet'], c=df_date['score_polarite'], cmap='coolwarm')
plt.clf()
plt.colorbar(plot)
# sns.color_palette("coolwarm", as_cmap=True)
# palette.reverse()
ax = sns.barplot(x=df_date.index, y=df_date['nbre_tweet'], hue=df_date['score_polarite'], palette='coolwarm', dodge=False)
# plt.xticks([df_date["date"][n] for n in np.arange(0, len(list(df_date["score_polarite"]))+1, 30)])
ax.xaxis.set_major_locator(matplotlib.ticker.MaxNLocator(prune='both', nbins=11))
ax.set(xlabel="Date",
       ylabel="nombre_tweets",
       title="Score de polarité moyen pour les tweets agrégés par date")
ax.legend_.remove()

In [ ]:
help(matplotlib.pyplot)

### Visualisation avec WordCloud

On se propose ici de visualiser les wordcloud selon le sentiment et selon le vaccin.

In [ ]:
def plot_wordcloud_label(label) :
    df_cloud = df_tweets[df_tweets['label']==label] 
    # join tweets to a single string
    words = ' '.join(df_cloud['texte_propre'])

    wordcloud = WordCloud(stopwords=stop_words,
                          background_color='white',
                          width=2000,
                          height=1500, max_words=100
                         ).generate(words)

    plt.figure(1,figsize=(10, 10))
    plt.imshow(wordcloud)
    plt.axis('off')
    plt.show()

In [ ]:
def plot_wordcloud_vaccin(vaccin) :
    df_cloud = df_tweets[df_tweets['type_vaccin']==vaccin] 
    # join tweets to a single string
    words = ' '.join(df_cloud['texte_propre'])

    wordcloud = WordCloud(stopwords=stop_words,
                          background_color='white',
                          width=2000,
                          height=1500, max_words=100
                         ).generate(words)

    plt.figure(1,figsize=(10, 10))
    plt.imshow(wordcloud)
    plt.axis('off')
    plt.show()

In [ ]:
plot_wordcloud_vaccin("

# Modélisation

Cette section concerne la modélisation où une analyse en composante principale et un modèle d'apprentissage supervisé seront implémentés. 

### Analyse en composante principale ACP

In [ ]:
# instanciation
sc = StandardScaler()
#transformation – centrage-réduction
Z = sc.fit_transform(X)
print(Z)

### Régression logistique

In [ ]:
dataframe = df_tweets[['texte', 'type_vaccin', 'nbre_like', 'nbre_retweet', 'score', 'nbre_hashtags', 'nbre_tags', 'nbre_mails', 'nbre_exclamation', 'nbre_interrogation', 'nbre_maj', 'label']]
X_train, X_test, y_train, y_test = train_test_split(dataframe.drop(['texte','label'], axis=1), df_tweets.label, test_size=0.2, random_state=12344)
logreg = LogisticRegression(max_iter=1000)
logreg.fit(X_train, y_train)
logreg.score(X_test, y_test)